In [1]:
## Load libraries
import pandas as pd
import numpy as np
import os
Main_Path = os.getcwd()

In [2]:
def Filter(file_name, period, ls_type):
    '''
    file_name: the name of the fundamental file
    period: length of the period used to do the selection (in months)
    This function would score the input stock
    Since we are trade in days, we would consider high daily trade volume (10M)
    with relatively large market capital (10B)
    '''
    fundamental = pd.read_csv(file_name, index_col=0).astype(float) ## Read data from file
    fundamental[fundamental > 1e+20] = 0 ## Set extrame large value as 0
    fundamental[fundamental < -1e+20] = 0 ## Set extrame small value as 0
    score = fundamental.rolling(period*30).mean().dropna()
    score['Volume'] = score['Volume'][score['Volume'] > 5000000] / 5000000.0
    score['Total Market Value'] = score['Total Market Value'][score['Total Market Value'] > 10000000000] / 10000000000.0
    if ls_type == 'long':
        score['P/E'] = score['P/E'][score['P/E'] > fundamental.loc[score.index]['P/E']]
        score['P/B'] = score['P/B'][score['P/B'] > fundamental.loc[score.index]['P/B']]
    else:
        score['P/E'] = score['P/E'][score['P/E'] < fundamental.loc[score.index]['P/E']]
        score['P/B'] = score['P/B'][score['P/B'] < fundamental.loc[score.index]['P/B']]
    score = score.fillna(0)
    score = 0.25*score['Volume'] + 0.2*score['Total Market Value'] + 0.3*score['P/E'] + 0.25*score['P/B']
    return score

In [3]:
## Load fundamental data and got 15 long stocks
path = Main_Path + '\\Fundamentals'
os.chdir(path)
for root, dirs, files in os.walk(path): ## Get all files under Fundamentals file
    fdmtl = files
Score_Map = pd.DataFrame(columns=[str(x).split('.')[0] for x in fdmtl])
for fd in fdmtl:
    Score_Map[fd.split('.')[0]] = Filter(fd, 1, 'long')
Score_Map_Rank = Score_Map.rank(axis=1, ascending=False)
Score_Map_Rank = Score_Map_Rank[Score_Map_Rank <= 15] ## If want to more stocks change 30, 31(next line) to any number
Chosen = pd.DataFrame(columns=[i for i in range(1, 16)], index=Score_Map_Rank.index)
for day in Score_Map_Rank.index:
    Chosen.loc[day] = Score_Map_Rank.loc[day].dropna().sort_values().index
os.chdir(Main_Path)
Chosen.to_csv('long_Chosen.csv')

In [4]:
## Load fundamental data and got 15 long stocks
path = Main_Path + '\\Fundamentals'
os.chdir(path)
for root, dirs, files in os.walk(path): ## Get all files under Fundamentals file
    fdmtl = files
Score_Map = pd.DataFrame(columns=[str(x).split('.')[0] for x in fdmtl])
for fd in fdmtl:
    Score_Map[fd.split('.')[0]] = Filter(fd, 1, 'short')
Score_Map_Rank = Score_Map.rank(axis=1, ascending=False)
Score_Map_Rank = Score_Map_Rank[Score_Map_Rank <= 15] ## If want to more stocks change 30, 31(next line) to any number
Chosen = pd.DataFrame(columns=[i for i in range(1, 16)], index=Score_Map_Rank.index)
for day in Score_Map_Rank.index:
    Chosen.loc[day] = Score_Map_Rank.loc[day].dropna().sort_values().index
os.chdir(Main_Path)
Chosen.to_csv('short_Chosen.csv')

In [8]:
## Combine both files
long = pd.read_csv('long_Chosen.csv', index_col=0)
short = pd.read_csv('short_Chosen.csv', index_col=0)
Chosen = pd.concat([long, short], axis=1)
Chosen.columns = [x for x in range(1, 31)]
Chosen.to_csv('30_Chosen.csv')